In [1]:
from qiskit import QuantumCircuit, Aer, transpile, IBMQ, providers
from qiskit.circuit import Parameter, ParameterVector

import numpy as np
import math

provider = IBMQ.load_account()

### Definition of circuits
Here both encoding and ansatz circuits are defined.
All circuits are made as circuits with unbound parameters 

In [ ]:
def encodeValidation(n,dataLenght, reps):
    """
    
    """
    if (dataLenght % n) > 0 and (dataLenght / n) > reps:
         raise Exception("nr of features and encoding circuit doesn't match") 
    else:
        pass

    
def qcAnsatz2(qubits, nrT):
    """"
    This function generates ansatz circuit number 2
    Inputs are number of qubits and the number of parameters
    
    :param int qubits: The number of qubits in the circuit
    :param int nrT: The number of optimisation parameters which should be generated
    :return: Ansatz circuit
    """
    
    paraPrQ = 2
    paraPrL = paraPrQ * qubits
    reps = math.ceil(nrT/paraPrL)
    encodeValidation(qubits, nrT, reps) #checks wether or not nrT and qubits match
    
    theta_list = ParameterVector('θ', length=reps*paraPrL)
    circuit = QuantumCircuit(qubits)
    for j in range(reps):
        for i in range(qubits):
            circuit.rx(theta_list[j*paraPrL+i],i)
            circuit.ry(theta_list[j*paraPrL+i+qubits],i)
        
        for i in range((qubits-1),0,-1):
            circuit.cx(i,i-1)
            
        #circuit.barrier()
    return circuit
    
    
    
def qcAnsatz9(qubits, nrT):
    """"
    This function generates ansatz circuit number 9
    Inputs are number of qubits and the number of parameters
    
    :param int qubits: The number of qubits in the circuit
    :param int nrT: The number of optimisation parameters which should be generated
    :return: Ansatz circuit
    """
    
    paraPrQ = 1
    paraPrL = paraPrQ * qubits
    reps = math.ceil(nrT/paraPrL)
    
    encodeValidation(qubits, nrT, reps) #checks wether or not nrT and qubits match
    
    theta_list = ParameterVector('θ', length=reps*qubits)
    circuit = QuantumCircuit(qubits)

    for j in range(reps):
        circuit.h(range(qubits))
        for i in range(qubits-1):
            circuit.swap(i,i+1)
        
        for i in range(qubits):
            circuit.rx(theta_list[j*qubits+i], i)
            
        #circuit.barrier()
    return circuit


def qcAnsatz14(qubits, nrT):
    
    """
    This function generates ansatz circuit number 14
    Inputs are number of qubits and the number of parameters
    
    :param int qubits: The number of qubits in the circuit
    :param int nrT: The number of optimisation parameters which should be generated
    :return: Ansatz circuit
    """
    
    paraPrQ = 4
    paraPrL = paraPrQ * qubits
    reps = math.ceil(nrT/paraPrL)
    
    encodeValidation(qubits, nrT, reps) #checks wether or not nrT and qubits match
    
    theta_list = ParameterVector('θ', length=reps*paraPrL)
    circuit = QuantumCircuit(qubits)
    counter = 0
    for j in range(reps):
        for i in range(qubits):
            #circuit.ry(theta_list[j*paraPrL+i], i)
            circuit.ry(theta_list[counter], i)
            counter += 1
        
        #circuit.barrier()
        #circuit.crx(theta_list[j*paraPrL+qubits+1], nrT-1, 0)
        circuit.crx(theta_list[counter], qubits-1, 0)
        counter += 1
       
        if qubits > 2:
            for i in range((qubits-2),-1,-1):
                circuit.crx(theta_list[counter],i,i+1)
                counter += 1
                
        #circuit.barrier()

        for i in range(qubits):
            circuit.ry(theta_list[counter], i)
            counter += 1
        
        #circuit.barrier()

        circuit.crx(theta_list[counter], qubits-1, qubits-2)
        counter += 1
        circuit.crx(theta_list[counter], 0, qubits-1,)
        counter += 1
        if qubits > 2:
            for i in range(1,(qubits-1)):
                circuit.crx(theta_list[counter], i, i-1,)
                counter += 1
        
        #circuit.barrier()     
    return circuit

### Code for transpilation

In [ ]:
nrQ = 5   #Qubits in circuits, should match number of classes
nrThetas = nrQ*24 #number of parameters to optimise

ansatz = qcAnsatz14(nrQ,nrThetas)


print('Depth:', ansatz.depth())
print('Gate counts:', ansatz.count_ops())
print()
transQC = []
Backend = provider.get_backend('ibm_oslo')


for kk in range(4):
    transQC.append(transpile(ansatz, Backend, optimization_level=kk))
    print('Optimization Level {}'.format(kk))
    print('Depth:', transQC[kk].depth())
    print('pr theta', transQC[kk].depth()/nrThetas)
    print('Gate counts:', sum(transQC[kk].count_ops().values()))
    print('pr theta', sum(transQC[kk].count_ops().values())/nrThetas)
    print()
    

In [ ]:
#Draw if you want to 
transQC[0].draw('mpl')

In [ ]:
from qiskit.tools.jupyter import *
%qiskit_version_table